In [1]:
import nbimporter
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity='all'
import tensorflow as tf 
import json
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import joblib
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import time
from sklearn.metrics import confusion_matrix
from utils import *
from sklearn.tree import DecisionTreeClassifier,plot_tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier 
from sklearn.ensemble import AdaBoostClassifier
from sklearn import svm,metrics,preprocessing,tree
from sklearn.ensemble import GradientBoostingClassifier

# performance of custom PWD training on WstPhish

In [3]:
 
wst_data=load_dataset('../data/wstphish/chspec_combine_western_full.json') 
data_wst=wst_data.iloc[:,:66] 
data_wst=data_wst.astype(float)
 
print(data_wst.shape)
 

(11204, 66)


# Custom Fc-based PWD training on WstPhish , 
1. f1 value refer to the red box on left side of Fig.8b, 
2. tpr and tnr refer to Table VIII, top cell of the left side,
3. te_time and tr_time refer to the left columns of Table VII

In [5]:
train_ze,test_ze=train_test_split(data_wst,test_size=0.2,shuffle=True)

train_ze_x=train_ze.iloc[:,0:65]
train_ze_y=train_ze.iloc[:,65] 
test_ze_x=test_ze.iloc[:,0:65] 
test_ze_y=test_ze.iloc[:,65] 
#print(test_ze.columns[49])
#print(test_ze.columns[0:49])

scaler_entire_ze=preprocessing.MinMaxScaler()
scaler_entire_ze.fit(train_ze_x)
train_x_ze=scaler_entire_ze.transform(train_ze_x)
test_x_ze=scaler_entire_ze.transform(test_ze_x)

#use only html features
scaler_html_ze=preprocessing.MinMaxScaler()
scaler_html_ze.fit(train_ze.iloc[:,36:65])
train_html_x_ze=scaler_html_ze.transform(train_ze.iloc[:,36:65])
test_html_x_ze=scaler_html_ze.transform(test_ze.iloc[:,36:65])
#print('train_ze html',train_ze.columns[25:49])
#only url features
scaler_url_ze=preprocessing.MinMaxScaler()
scaler_url_ze.fit(train_ze.iloc[:,0:36])
#print('train ch url is',train_ze.columns[0:25])

train_url_x_ze=scaler_url_ze.transform(train_ze.iloc[:,0:36])
test_url_x_ze=scaler_url_ze.transform(test_ze.iloc[:,0:36])

test_url_y_ze=test_html_y_ze=test_ze_y
train_url_y_ze=train_html_y_ze=train_ze_y

print('combine_rf_whole:')# train_ze_x

rf_start=time.time()
model_rf1_ze=RandomForestClassifier(criterion='gini', max_depth=55, min_samples_split=3, n_estimators=200)

model_rf_com,rf_tr_time,rf_te_time,rf_tnr_com,rf_recall_com,rf_f1_com=time_metri(model_rf1_ze,train_x_ze,train_ze_y,test_x_ze,test_ze_y)#using normalized data to train, so all the model_rf4_com data must normalized

print('combine_lr_whole:')
model_lr1_ze = LogisticRegression(max_iter=500,penalty='l2',C=10)

model_lr_com,lr_tr_time,lr_te_time,lr_tnr_com,lr_recall_com,lr_f1_com =time_metri(model_lr1_ze,train_x_ze,train_ze_y,test_x_ze,test_ze_y)
model_dt1_ze=DecisionTreeClassifier(max_depth=24, min_samples_leaf= 1, min_samples_split=5)
print('dt======')
model_dt_com,dt_tr_time,dt_te_time,dt_tnr_com,dt_recall_com,dt_f1_com =time_metri(model_dt1_ze,train_x_ze,train_ze_y,test_x_ze,test_ze_y)
model_knn1_ze = KNeighborsClassifier(n_neighbors=3)

print('knn======')
model_knn_com,knn_tr_time,knn_te_time,knn_tnr_com,knn_recall_com,knn_f1_com= time_metri(model_knn1_ze,train_x_ze,train_ze_y,test_x_ze,test_ze_y)
print('mlp======')
model_mlp1_ze=MLPClassifier(activation='relu', max_iter=1500, solver='lbfgs',learning_rate='adaptive')
model_mlp_com,mlp_tr_time,mlp_te_time,mlp_tnr_com,mlp_recall_com,mlp_f1_com=time_metri(model_mlp1_ze,train_x_ze,train_ze_y,test_x_ze,test_ze_y)

print('ab====')
model_ada_ze=AdaBoostClassifier(learning_rate=1.2,n_estimators=200)

model_ada_com,ab_tr_time,ab_te_time,ab_tnr_com,ab_recall_com,ab_f1_com=time_metri(model_ada_ze,train_x_ze,train_ze_y,test_x_ze,test_ze_y)
print('combine svm whole:')

model_svm_ze=svm.SVC(C=50, gamma=1, kernel= 'linear')
model_svm_com,svm_tr_time,svm_te_time,svm_tnr_com,svm_recall_com,svm_f1_com=time_metri(model_svm_ze,train_x_ze,train_ze_y,test_x_ze,test_ze_y)
print('gb======')
model_gb_ze=GradientBoostingClassifier(learning_rate= 0.2, n_estimators= 300)

model_gb_com,gb_tr_time,gb_te_time,gb_tnr_com,gb_recall_com,gb_f1_com=time_metri(model_gb_ze,train_x_ze,train_ze_y,test_x_ze,test_ze_y)


# for cnn models
one_hot_train_y_ze=tf.one_hot(indices=train_ze_y.array,depth=2,on_value=1.0,off_value=0.0)
one_hot_test_y_ze=tf.one_hot(indices=test_ze_y.array,depth=2,on_value=1.0,off_value=0.0)
test_x_re_ze=np.asarray(test_x_ze).reshape(test_x_ze.shape[0],test_x_ze.shape[1],1)
train_x_re_ze=np.asarray(train_x_ze).reshape(train_x_ze.shape[0],train_x_ze.shape[1],1)
train_html_x_re_ze=np.asarray(train_html_x_ze).reshape(train_html_x_ze.shape[0],train_html_x_ze.shape[1],1)
test_html_x_re_ze=np.asarray(test_html_x_ze).reshape(test_html_x_ze.shape[0],test_html_x_ze.shape[1],1)
train_url_x_re_ze=np.asarray(train_url_x_ze).reshape(train_url_x_ze.shape[0],train_url_x_ze.shape[1],1)
test_url_x_re_ze=np.asarray(test_url_x_ze).reshape(test_url_x_ze.shape[0],test_url_x_ze.shape[1],1)
 
print('cnn======')

model_cnn_combined=create_model_cnn_full()
cnn_start=time.time()
model_cnn_combined.fit(train_x_re_ze,one_hot_train_y_ze,batch_size=32,epochs=200, verbose=2)
cnn_end=time.time()
cnn_tr_time=cnn_end-cnn_start

predict_y_com=model_cnn_combined.predict(test_x_re_ze)
pre_class_com=np.argmax(predict_y_com,axis=1)
confusion_mtx_com = confusion_matrix(test_ze_y, pre_class_com) 
cnn_tnr_com,cnn_recall_com,cnn_f1_com=get_fpr(test_ze_y,pre_class_com)
print('tpr is %.2f, tnr is %.2f, f1 is %.2f'%(cnn_recall_com,cnn_tnr_com,cnn_f1_com))
cnn_te_time=time.time()-cnn_end
print('cnn_tr_time is',cnn_tr_time)
print('cnn_te_time is',cnn_te_time)


MinMaxScaler()

MinMaxScaler()

MinMaxScaler()

combine_rf_whole:
te_time is 0.0593
tr_time is 1.1640
tpr is 0.99, tnr is 0.97,f1 is 0.98
combine_lr_whole:
te_time is 0.0003
tr_time is 0.3761
tpr is 0.95, tnr is 0.95,f1 is 0.96
dt======
te_time is 0.0005
tr_time is 0.0629
tpr is 0.98, tnr is 0.95,f1 is 0.98
knn======
te_time is 0.3052
tr_time is 0.0013
tpr is 0.95, tnr is 0.93,f1 is 0.96
mlp======
te_time is 0.0013
tr_time is 6.7719
tpr is 0.97, tnr is 0.96,f1 is 0.97
ab====
te_time is 0.0823
tr_time is 1.6866
tpr is 0.97, tnr is 0.96,f1 is 0.98
combine svm whole:
te_time is 0.0560
tr_time is 2.4752
tpr is 0.95, tnr is 0.95,f1 is 0.96
gb======
te_time is 0.0095
tr_time is 4.4878
tpr is 0.99, tnr is 0.98,f1 is 0.99
cnn======
Epoch 1/200
281/281 - 4s - loss: 0.4045 - categorical_accuracy: 0.8044 - 4s/epoch - 14ms/step
Epoch 2/200
281/281 - 4s - loss: 0.2223 - categorical_accuracy: 0.9090 - 4s/epoch - 13ms/step
Epoch 3/200
281/281 - 4s - loss: 0.1792 - categorical_accuracy: 0.9296 - 4s/epoch - 13ms/step
Epoch 4/200
281/281 - 4s - loss:

281/281 - 3s - loss: 0.0364 - categorical_accuracy: 0.9871 - 3s/epoch - 12ms/step
Epoch 81/200
281/281 - 3s - loss: 0.0394 - categorical_accuracy: 0.9864 - 3s/epoch - 12ms/step
Epoch 82/200
281/281 - 3s - loss: 0.0327 - categorical_accuracy: 0.9883 - 3s/epoch - 12ms/step
Epoch 83/200
281/281 - 3s - loss: 0.0382 - categorical_accuracy: 0.9852 - 3s/epoch - 12ms/step
Epoch 84/200
281/281 - 3s - loss: 0.0330 - categorical_accuracy: 0.9875 - 3s/epoch - 12ms/step
Epoch 85/200
281/281 - 3s - loss: 0.0364 - categorical_accuracy: 0.9863 - 3s/epoch - 12ms/step
Epoch 86/200
281/281 - 3s - loss: 0.0349 - categorical_accuracy: 0.9861 - 3s/epoch - 12ms/step
Epoch 87/200
281/281 - 3s - loss: 0.0322 - categorical_accuracy: 0.9882 - 3s/epoch - 12ms/step
Epoch 88/200
281/281 - 3s - loss: 0.0338 - categorical_accuracy: 0.9868 - 3s/epoch - 12ms/step
Epoch 89/200
281/281 - 4s - loss: 0.0348 - categorical_accuracy: 0.9856 - 4s/epoch - 13ms/step
Epoch 90/200
281/281 - 4s - loss: 0.0344 - categorical_accuracy

Epoch 166/200
281/281 - 3s - loss: 0.0210 - categorical_accuracy: 0.9925 - 3s/epoch - 12ms/step
Epoch 167/200
281/281 - 3s - loss: 0.0273 - categorical_accuracy: 0.9907 - 3s/epoch - 12ms/step
Epoch 168/200
281/281 - 3s - loss: 0.0225 - categorical_accuracy: 0.9913 - 3s/epoch - 12ms/step
Epoch 169/200
281/281 - 3s - loss: 0.0192 - categorical_accuracy: 0.9936 - 3s/epoch - 12ms/step
Epoch 170/200
281/281 - 3s - loss: 0.0217 - categorical_accuracy: 0.9923 - 3s/epoch - 12ms/step
Epoch 171/200
281/281 - 3s - loss: 0.0156 - categorical_accuracy: 0.9941 - 3s/epoch - 12ms/step
Epoch 172/200
281/281 - 3s - loss: 0.0189 - categorical_accuracy: 0.9922 - 3s/epoch - 12ms/step
Epoch 173/200
281/281 - 3s - loss: 0.0210 - categorical_accuracy: 0.9927 - 3s/epoch - 12ms/step
Epoch 174/200
281/281 - 3s - loss: 0.0186 - categorical_accuracy: 0.9933 - 3s/epoch - 12ms/step
Epoch 175/200
281/281 - 3s - loss: 0.0164 - categorical_accuracy: 0.9945 - 3s/epoch - 12ms/step
Epoch 176/200
281/281 - 3s - loss: 0.020

cnn_tr_time is 694.5239400863647
cnn_te_time is 0.3825826644897461


# Fu- and Fh-based PWD
1. Fu-based PWD training on WstPhish, refer to top cell of the left column on Table X,
2. Fh-based PWD training on WstPhish, refer to top cell of the left column on Table IX

In [9]:
model_rf_ze_url=RandomForestClassifier(criterion='gini', max_depth=50, min_samples_split=3, n_estimators=120)
model_rf_ze_html=RandomForestClassifier(criterion='entropy', max_depth=50, min_samples_split=3, n_estimators=160)

print('only_url_rf_test:')
model_rf_url,rf_tnr_url,rf_recall_url,rf_time_url,rf_f1_url=full_metri(model_rf_ze_url,train_url_x_ze,train_url_y_ze,test_url_x_ze,test_url_y_ze)

print('only_html_rf_test:')
model_rf_html,rf_tnr_html,rf_recall_html,rf_time_html,rf_f1_html=full_metri(model_rf_ze_html,train_html_x_ze,train_html_y_ze,test_html_x_ze,test_html_y_ze)
model_lr_ze_url = LogisticRegression(max_iter=500,penalty='l2',C=100)
model_lr_ze_html = LogisticRegression(max_iter=500,penalty='l2',C=0.01)

print('only_url_lr_whole_test:')
model_lr_url,lr_tnr_url,lr_recall_url,lr_time_url,lr_f1_url=full_metri(model_lr_ze_url,train_url_x_ze,train_url_y_ze,test_url_x_ze,test_url_y_ze)

print('only_html_lr_whole_test:')
model_lr_html,lr_tnr_html,lr_recall_html,lr_time_html,lr_f1_html=full_metri(model_lr_ze_html,train_html_x_ze,train_html_y_ze,test_html_x_ze,test_html_y_ze)


model_dt_ze_url=DecisionTreeClassifier(max_depth=34, min_samples_leaf= 1, min_samples_split=2)
model_dt_ze_html=DecisionTreeClassifier(max_depth=28, min_samples_leaf= 1, min_samples_split=2)
print('only_url_dt_test:')
model_dt_url,dt_tnr_url,dt_recall_url,dt_time_url,dt_f1_url=full_metri(model_dt_ze_url,train_url_x_ze,train_url_y_ze,test_url_x_ze,test_url_y_ze)
#only html
print('only_html_dt_test:')
model_dt_html,dt_tnr_html,dt_recall_html,dt_time_html,dt_f1_html=full_metri(model_dt_ze_html,train_html_x_ze,train_html_y_ze,test_html_x_ze,test_html_y_ze)

model_knn2_ze = KNeighborsClassifier(n_neighbors=3)
model_knn3_ze = KNeighborsClassifier(n_neighbors=5)
print('only_url_knn_test:')
model_knn_url,knn_tnr_url,knn_recall_url,knn_time_url,knn_f1_url=full_metri(model_knn2_ze,train_url_x_ze,train_url_y_ze,test_url_x_ze,test_url_y_ze)
print('only_html_knn_test:')
model_knn_html,knn_tnr_html,knn_recall_html,knn_time_html,knn_f1_html=full_metri(model_knn3_ze,train_html_x_ze,train_html_y_ze,test_html_x_ze,test_html_y_ze)


model_mlp2_ze=MLPClassifier(activation='relu', max_iter=2500, solver='adam',learning_rate='adaptive')
model_mlp_ze_html=MLPClassifier(activation='relu', max_iter=1500, solver='adam',learning_rate='adaptive')
print('only_url_mlp_test:')
model_mlp_url,mlp_tnr_url,mlp_recall_url,mlp_time_url,mlp_f1_url=full_metri(model_mlp2_ze,train_url_x_ze,train_url_y_ze,test_url_x_ze,test_url_y_ze)
print('only_html_mlp_test:')
model_mlp_html,mlp_tnr_html,mlp_recall_html,mlp_time_html,mlp_f1_html=full_metri(model_mlp_ze_html,train_html_x_ze,train_html_y_ze,test_html_x_ze,test_html_y_ze)



model_gb_ze_url=GradientBoostingClassifier(learning_rate=0.15, n_estimators= 200)
model_gb_ze_html=GradientBoostingClassifier(learning_rate=0.2, n_estimators= 300)


print('only_url_gb_test:')
model_gb_url,gb_tnr_url,gb_recall_url,gb_time_url,gb_f1_url=full_metri(model_gb_ze_url,train_url_x_ze,train_url_y_ze,test_url_x_ze,test_url_y_ze)

print('only_html_gb_test:')
model_gb_html,gb_tnr_html,gb_recall_html,gb_time_html,gb_f1_html=full_metri(model_gb_ze_html,train_html_x_ze,train_html_y_ze,test_html_x_ze,test_html_y_ze)


model_ada_ze_url=AdaBoostClassifier(learning_rate=0.9,n_estimators=300)
model_ada_ze_html=AdaBoostClassifier(learning_rate=1.5, n_estimators=300)
print('only_url_ab_test:')
model_ada_url,ada_tnr_url,ada_recall_url,ada_time_url,ada_f1_url=full_metri(model_ada_ze_url,train_url_x_ze,train_url_y_ze,test_url_x_ze,test_url_y_ze)
print('only_html_ab_test:')
model_ada_html,ada_tnr_html,ada_recall_html,ada_time_html,ada_f1_html=full_metri(model_ada_ze_html,train_html_x_ze,train_html_y_ze,test_html_x_ze,test_html_y_ze)

model_svm_ze_url=svm.SVC(C=100, gamma=1, kernel='linear')
model_svm_ze_html=svm.SVC(C=50, gamma=1, kernel='linear')

print('only_url_svm_test:')
model_svm_url,svm_tnr_url,svm_recall_url,svm_time_url,svm_f1_url=full_metri(model_svm_ze_url,train_url_x_ze,train_url_y_ze,test_url_x_ze,test_url_y_ze)

print('only_html_svm_test:')
model_svm_html,svm_tnr_html,svm_recall_html,svm_time_html,svm_f1_html=full_metri(model_svm_ze_html,train_html_x_ze,train_html_y_ze,test_html_x_ze,test_html_y_ze)

# for cnn models
print('cnn_url:') 
model_cnn_url=create_model_url_full()
#cnn_url_tr_start=time.time()
model_cnn_url.fit(train_url_x_re_ze,one_hot_train_y_ze,batch_size=32,epochs=200, verbose=2)

predict_y_url=model_cnn_url.predict(test_url_x_re_ze)
pre_class_url=np.argmax(predict_y_url,axis=1)
confusion_mtx_url = confusion_matrix(test_url_y_ze, pre_class_url) 
#print(confusion_mtx_com)
cnn_tnr_url,cnn_recall_url,cnn_f1_url=get_fpr(test_url_y_ze,pre_class_url)
print('tpr is %.2f, tnr is %.2f'%(cnn_recall_url,cnn_tnr_url))

print('cnn_html:')
model_cnn_html=create_model_html_full()
#cnn_html_tr_start=time.time()
model_cnn_html.fit(train_html_x_re_ze,one_hot_train_y_ze,batch_size=32,epochs=200, verbose=2) 

predict_y_html=model_cnn_html.predict(test_html_x_re_ze)
pre_class_html=np.argmax(predict_y_html,axis=1)
confusion_mtx_html = confusion_matrix(test_html_y_ze, pre_class_html) 
#print(confusion_mtx_com)
cnn_tnr_html,cnn_recall_html,cnn_f1_html=get_fpr(test_html_y_ze,pre_class_html)
print('tpr is %.2f, tnr is %.2f'%(cnn_recall_html,cnn_tnr_html))
   

only_url_rf_test:
recall is 0.98,tnr is 0.96
only_html_rf_test:
recall is 0.95,tnr is 0.93
only_url_dt_test:
recall is 0.98,tnr is 0.94
only_html_dt_test:
recall is 0.94,tnr is 0.83
only_url_knn_test:
recall is 0.94,tnr is 0.94
only_html_knn_test:
recall is 0.92,tnr is 0.83
only_url_mlp_test:
recall is 0.97,tnr is 0.96
only_html_mlp_test:
recall is 0.93,tnr is 0.84
only_url_gb_test:
recall is 0.98,tnr is 0.97
only_html_gb_test:
recall is 0.93,tnr is 0.86
only_url_ab_test:
recall is 0.97,tnr is 0.95
only_html_ab_test:
recall is 0.91,tnr is 0.78
only_url_svm_test:
recall is 0.95,tnr is 0.95
only_html_svm_test:
recall is 0.86,tnr is 0.76
cnn_url:
Epoch 1/200
281/281 - 3s - loss: 0.4278 - categorical_accuracy: 0.7900 - 3s/epoch - 10ms/step
Epoch 2/200
281/281 - 2s - loss: 0.2441 - categorical_accuracy: 0.9033 - 2s/epoch - 8ms/step
Epoch 3/200
281/281 - 2s - loss: 0.2030 - categorical_accuracy: 0.9203 - 2s/epoch - 8ms/step
Epoch 4/200
281/281 - 2s - loss: 0.1884 - categorical_accuracy: 0.92

Epoch 82/200
281/281 - 2s - loss: 0.0780 - categorical_accuracy: 0.9703 - 2s/epoch - 8ms/step
Epoch 83/200
281/281 - 2s - loss: 0.0751 - categorical_accuracy: 0.9715 - 2s/epoch - 8ms/step
Epoch 84/200
281/281 - 2s - loss: 0.0774 - categorical_accuracy: 0.9714 - 2s/epoch - 8ms/step
Epoch 85/200
281/281 - 2s - loss: 0.0758 - categorical_accuracy: 0.9703 - 2s/epoch - 8ms/step
Epoch 86/200
281/281 - 2s - loss: 0.0751 - categorical_accuracy: 0.9720 - 2s/epoch - 8ms/step
Epoch 87/200
281/281 - 2s - loss: 0.0761 - categorical_accuracy: 0.9702 - 2s/epoch - 8ms/step
Epoch 88/200
281/281 - 2s - loss: 0.0798 - categorical_accuracy: 0.9704 - 2s/epoch - 8ms/step
Epoch 89/200
281/281 - 2s - loss: 0.0720 - categorical_accuracy: 0.9705 - 2s/epoch - 8ms/step
Epoch 90/200
281/281 - 2s - loss: 0.0757 - categorical_accuracy: 0.9711 - 2s/epoch - 8ms/step
Epoch 91/200
281/281 - 2s - loss: 0.0760 - categorical_accuracy: 0.9715 - 2s/epoch - 8ms/step
Epoch 92/200
281/281 - 2s - loss: 0.0739 - categorical_accur

Epoch 169/200
281/281 - 2s - loss: 0.0636 - categorical_accuracy: 0.9751 - 2s/epoch - 8ms/step
Epoch 170/200
281/281 - 2s - loss: 0.0611 - categorical_accuracy: 0.9772 - 2s/epoch - 8ms/step
Epoch 171/200
281/281 - 2s - loss: 0.0599 - categorical_accuracy: 0.9763 - 2s/epoch - 8ms/step
Epoch 172/200
281/281 - 2s - loss: 0.0642 - categorical_accuracy: 0.9748 - 2s/epoch - 8ms/step
Epoch 173/200
281/281 - 2s - loss: 0.0607 - categorical_accuracy: 0.9780 - 2s/epoch - 8ms/step
Epoch 174/200
281/281 - 2s - loss: 0.0638 - categorical_accuracy: 0.9759 - 2s/epoch - 8ms/step
Epoch 175/200
281/281 - 2s - loss: 0.0617 - categorical_accuracy: 0.9778 - 2s/epoch - 8ms/step
Epoch 176/200
281/281 - 2s - loss: 0.0591 - categorical_accuracy: 0.9771 - 2s/epoch - 8ms/step
Epoch 177/200
281/281 - 2s - loss: 0.0621 - categorical_accuracy: 0.9770 - 2s/epoch - 8ms/step
Epoch 178/200
281/281 - 2s - loss: 0.0653 - categorical_accuracy: 0.9759 - 2s/epoch - 8ms/step
Epoch 179/200
281/281 - 2s - loss: 0.0601 - catego

tpr is 0.98, tnr is 0.97
cnn_html:
Epoch 1/200
281/281 - 3s - loss: 0.4917 - categorical_accuracy: 0.7648 - 3s/epoch - 11ms/step
Epoch 2/200
281/281 - 3s - loss: 0.4162 - categorical_accuracy: 0.8143 - 3s/epoch - 9ms/step
Epoch 3/200
281/281 - 3s - loss: 0.3934 - categorical_accuracy: 0.8247 - 3s/epoch - 9ms/step
Epoch 4/200
281/281 - 3s - loss: 0.3793 - categorical_accuracy: 0.8296 - 3s/epoch - 9ms/step
Epoch 5/200
281/281 - 3s - loss: 0.3613 - categorical_accuracy: 0.8386 - 3s/epoch - 9ms/step
Epoch 6/200
281/281 - 2s - loss: 0.3575 - categorical_accuracy: 0.8448 - 2s/epoch - 8ms/step
Epoch 7/200
281/281 - 2s - loss: 0.3486 - categorical_accuracy: 0.8484 - 2s/epoch - 8ms/step
Epoch 8/200
281/281 - 2s - loss: 0.3342 - categorical_accuracy: 0.8557 - 2s/epoch - 8ms/step
Epoch 9/200
281/281 - 3s - loss: 0.3268 - categorical_accuracy: 0.8586 - 3s/epoch - 9ms/step
Epoch 10/200
281/281 - 3s - loss: 0.3223 - categorical_accuracy: 0.8623 - 3s/epoch - 9ms/step
Epoch 11/200
281/281 - 2s - loss:

Epoch 88/200
281/281 - 2s - loss: 0.2018 - categorical_accuracy: 0.9211 - 2s/epoch - 9ms/step
Epoch 89/200
281/281 - 3s - loss: 0.2035 - categorical_accuracy: 0.9162 - 3s/epoch - 9ms/step
Epoch 90/200
281/281 - 2s - loss: 0.1977 - categorical_accuracy: 0.9245 - 2s/epoch - 6ms/step
Epoch 91/200
281/281 - 2s - loss: 0.1986 - categorical_accuracy: 0.9244 - 2s/epoch - 9ms/step
Epoch 92/200
281/281 - 3s - loss: 0.2008 - categorical_accuracy: 0.9231 - 3s/epoch - 9ms/step
Epoch 93/200
281/281 - 2s - loss: 0.1990 - categorical_accuracy: 0.9232 - 2s/epoch - 8ms/step
Epoch 94/200
281/281 - 2s - loss: 0.1988 - categorical_accuracy: 0.9222 - 2s/epoch - 9ms/step
Epoch 95/200
281/281 - 3s - loss: 0.1978 - categorical_accuracy: 0.9250 - 3s/epoch - 9ms/step
Epoch 96/200
281/281 - 3s - loss: 0.2000 - categorical_accuracy: 0.9232 - 3s/epoch - 9ms/step
Epoch 97/200
281/281 - 3s - loss: 0.1918 - categorical_accuracy: 0.9248 - 3s/epoch - 9ms/step
Epoch 98/200
281/281 - 3s - loss: 0.1947 - categorical_accur

Epoch 175/200
281/281 - 2s - loss: 0.1640 - categorical_accuracy: 0.9374 - 2s/epoch - 9ms/step
Epoch 176/200
281/281 - 3s - loss: 0.1762 - categorical_accuracy: 0.9323 - 3s/epoch - 9ms/step
Epoch 177/200
281/281 - 2s - loss: 0.1633 - categorical_accuracy: 0.9351 - 2s/epoch - 9ms/step
Epoch 178/200
281/281 - 2s - loss: 0.1691 - categorical_accuracy: 0.9355 - 2s/epoch - 9ms/step
Epoch 179/200
281/281 - 3s - loss: 0.1672 - categorical_accuracy: 0.9346 - 3s/epoch - 9ms/step
Epoch 180/200
281/281 - 3s - loss: 0.1678 - categorical_accuracy: 0.9366 - 3s/epoch - 9ms/step
Epoch 181/200
281/281 - 3s - loss: 0.1615 - categorical_accuracy: 0.9381 - 3s/epoch - 9ms/step
Epoch 182/200
281/281 - 3s - loss: 0.1625 - categorical_accuracy: 0.9373 - 3s/epoch - 9ms/step
Epoch 183/200
281/281 - 3s - loss: 0.1658 - categorical_accuracy: 0.9346 - 3s/epoch - 9ms/step
Epoch 184/200
281/281 - 2s - loss: 0.1699 - categorical_accuracy: 0.9356 - 2s/epoch - 9ms/step
Epoch 185/200
281/281 - 2s - loss: 0.1660 - catego

tpr is 0.93, tnr is 0.89


# WstPhish PWD predict ChiPhish, refer to top cell of right column on Table VIII, IX, X

In [37]:
ch_feature='../data/chiphish/chspec_combine_chphish_full.json' 
ch_data=load_dataset(ch_feature)
ch_data.shape  
ch_data['label'].value_counts()
data_ch=ch_data.iloc[:,:66]  
data_ch=data_ch.astype(float)
 

(1620, 69)

0    1055
1     565
Name: label, dtype: int64

In [51]:

train_ch,test_ch=train_test_split(data_ch,test_size=0.2,shuffle=True)

pre_html_x_ch=test_ch.iloc[:,36:65]
pre_html_y_ch=pre_url_y_ch=pre_y_ch=test_ch.iloc[:,65]
pre_url_x_ch=test_ch.iloc[:,0:36]
pre_x_ch=test_ch.iloc[:,0:65]

#     scaler_entire_ze=joblib.load('./zenodo_model_spec/scaler_entire_ze_'+str(i)+'.pkl')
#     #use only html features
#     scaler_html_ze=joblib.load('./zenodo_model_spec/scaler_html_ze_'+str(i)+'.pkl')
#     #only url features
#     scaler_url_ze=joblib.load('./zenodo_model_spec/scaler_url_ze_'+str(i)+'.pkl')
#     # for classifier
pre_x_ch=scaler_entire_ze.transform(pre_x_ch)
pre_url_x_ch=scaler_url_ze.transform(pre_url_x_ch)
pre_html_x_ch=scaler_html_ze.transform(pre_html_x_ch)

# for cnn
one_hot_pre_y_ch=tf.one_hot(indices=pre_y_ch.array,depth=2,on_value=1.0,off_value=0.0)
pre_x_re_ch=np.asarray(pre_x_ch).reshape(pre_x_ch.shape[0],pre_x_ch.shape[1],1)
pre_html_x_re_ch=np.asarray(pre_html_x_ch).reshape(pre_html_x_ch.shape[0],pre_html_x_ch.shape[1],1)
pre_url_x_re_ch=np.asarray(pre_url_x_ch).reshape(pre_url_x_ch.shape[0],pre_url_x_ch.shape[1],1)

print('mlp=======')
mlp_tnr_com,mlp_recall_com,mlp_f1_com=pre_metri(model_mlp1_ze,pre_x_ch,pre_y_ch)
mlp_tnr_url,mlp_recall_url,mlp_f1_url=pre_metri(model_mlp2_ze,pre_url_x_ch,pre_url_y_ch)
mlp_tnr_html,mlp_recall_html,mlp_f1_html=pre_metri(model_mlp_ze_html,pre_html_x_ch,pre_html_y_ch)

print('rf========')
rf_tnr_com,rf_recall_com,rf_f1_com=pre_metri(model_rf1_ze,pre_x_ch,pre_y_ch)
rf_tnr_url,rf_recall_url,rf_f1_url=pre_metri(model_rf_ze_url,pre_url_x_ch,pre_url_y_ch)
rf_tnr_html,rf_recall_html,rf_f1_html=pre_metri(model_rf_ze_html,pre_html_x_ch,pre_html_y_ch)

print('lr=========')
lr_tnr_com,lr_recall_com,lr_f1_com=pre_metri(model_lr1_ze,pre_x_ch,pre_y_ch)
lr_tnr_url,lr_recall_url,lr_f1_url=pre_metri(model_lr_ze_url,pre_url_x_ch,pre_url_y_ch)
lr_tnr_html,lr_recall_html,lr_f1_html=pre_metri(model_lr_ze_html,pre_html_x_ch,pre_html_y_ch)
print('dt========')
dt_tnr_com,dt_recall_com,dt_f1_com=pre_metri(model_dt1_ze,pre_x_ch,pre_y_ch)
dt_tnr_url,dt_recall_url,dt_f1_url=pre_metri(model_dt_ze_url,pre_url_x_ch,pre_url_y_ch)
dt_tnr_html,dt_recall_html,dt_f1_html=pre_metri(model_dt_ze_html,pre_html_x_ch,pre_html_y_ch)

print('knn=======')
knn_tnr_com,knn_recall_com,knn_f1_com=pre_metri(model_knn1_ze,pre_x_ch,pre_y_ch)
knn_tnr_url,knn_recall_url,knn_f1_url=pre_metri(model_knn2_ze,pre_url_x_ch,pre_url_y_ch)
knn_tnr_html,knn_recall_html,knn_f1_html=pre_metri(model_knn3_ze,pre_html_x_ch,pre_html_y_ch)

print('gb=========')
gb_tnr_com,gb_recall_com,gb_f1_com=pre_metri(model_gb_ze,pre_x_ch,pre_y_ch)
gb_tnr_url,gb_recall_url,gb_f1_url=pre_metri(model_gb_ze_url,pre_url_x_ch,pre_url_y_ch)
gb_tnr_html,gb_recall_html,gb_f1_html=pre_metri(model_gb_ze_html,pre_html_x_ch,pre_html_y_ch)

print('ab========')
ada_tnr_com,ada_recall_com,ada_f1_com=pre_metri(model_ada_ze,pre_x_ch,pre_y_ch)
ada_tnr_url,ada_recall_url,ada_f1_url=pre_metri(model_ada_ze_url,pre_url_x_ch,pre_url_y_ch)
ada_tnr_html,ada_recall_html,ada_f1_html=pre_metri(model_ada_ze_html,pre_html_x_ch,pre_html_y_ch)

print('svm=======')
svm_tnr_com,svm_recall_com,svm_f1_com=pre_metri(model_svm_ze,pre_x_ch,pre_y_ch)
svm_tnr_url,svm_recall_url,svm_f1_url=pre_metri(model_svm_ze_url,pre_url_x_ch,pre_url_y_ch)
svm_tnr_html,svm_recall_html,svm_f1_html=pre_metri(model_ada_ze_html,pre_html_x_ch,pre_html_y_ch)


print('cnn=====')
cnn_tnr_com,cnn_recall_com,cnn_f1_com=cnn_metri(model_cnn_combined,pre_x_re_ch,pre_y_ch)
print('tpr is %.2f, tnr is %.2f'%(cnn_recall_com,cnn_tnr_com))
cnn_tnr_url,cnn_recall_url,cnn_f1_url=cnn_metri(model_cnn_url,pre_url_x_re_ch,pre_y_ch)
print('tpr is %.2f, tnr is %.2f'%(cnn_recall_url,cnn_tnr_url))
cnn_tnr_html,cnn_recall_html,cnn_f1_html=cnn_metri(model_cnn_html,pre_html_x_re_ch,pre_y_ch)
print('tpr is %.2f, tnr is %.2f'%(cnn_recall_html,cnn_tnr_html))


    
    

mlp=======
recall is 0.32, tnr is 0.85
recall is 0.46, tnr is 0.87
recall is 0.29, tnr is 0.63
rf========
recall is 0.36, tnr is 0.99
recall is 0.51, tnr is 0.93
recall is 0.13, tnr is 0.95
lr=========
recall is 0.45, tnr is 0.96
recall is 0.72, tnr is 0.81
recall is 0.37, tnr is 0.87
dt========
recall is 0.54, tnr is 0.84
recall is 0.74, tnr is 0.70
recall is 0.33, tnr is 0.84
knn=======
recall is 0.21, tnr is 0.97
recall is 0.57, tnr is 0.88
recall is 0.11, tnr is 0.96
gb=========
recall is 0.59, tnr is 0.96
recall is 0.74, tnr is 0.85
recall is 0.26, tnr is 0.97
ab========
recall is 0.71, tnr is 0.78
recall is 0.79, tnr is 0.67
recall is 0.09, tnr is 0.95
svm=======
recall is 0.49, tnr is 0.93
recall is 0.70, tnr is 0.85
recall is 0.09, tnr is 0.95
cnn=====
recall is 0.38, tnr is 0.92
tpr is 0.38, tnr is 0.92
recall is 0.62, tnr is 0.68
tpr is 0.62, tnr is 0.68
recall is 0.13, tnr is 0.93
tpr is 0.13, tnr is 0.93


# WstPhish PWD predict EngPhish, refer to top cell of middle column on Table VIII, IX, X

In [11]:
en_feature='../data/engphish/chspec_combine_enphish_full.json' 
en_data=load_dataset(en_feature)
en_data.shape  
en_data['label'].value_counts()

data_en=en_data.iloc[:,:66]#include 'label', 'index', 
data_en=data_en.astype(float)
print('data_en',data_en.shape)
# print(data_en.columns)

(15111, 69)

0    11019
1     4092
Name: label, dtype: int64

data_en (15111, 66)


In [35]:
train_en,test_en=train_test_split(data_en,test_size=0.2,shuffle=True)

pre_html_x_en=test_en.iloc[:,36:65]
pre_html_y_en=pre_url_y_en=pre_y_en=test_en.iloc[:,65]
pre_url_x_en=test_en.iloc[:,0:36]
pre_x_en=test_en.iloc[:,0:65]

pre_x_en=scaler_entire_ze.transform(pre_x_en)
pre_url_x_en=scaler_url_ze.transform(pre_url_x_en)
pre_html_x_en=scaler_html_ze.transform(pre_html_x_en)

# for cnn
one_hot_pre_y_en=tf.one_hot(indices=pre_y_en.array,depth=2,on_value=1.0,off_value=0.0)
pre_x_re_en=np.asarray(pre_x_en).reshape(pre_x_en.shape[0],pre_x_en.shape[1],1)
pre_html_x_re_en=np.asarray(pre_html_x_en).reshape(pre_html_x_en.shape[0],pre_html_x_en.shape[1],1)
pre_url_x_re_en=np.asarray(pre_url_x_en).reshape(pre_url_x_en.shape[0],pre_url_x_en.shape[1],1)

print('mlp======')
mlp_tnr_com,mlp_recall_com,mlp_f1_com=pre_metri(model_mlp1_ze,pre_x_en,pre_y_en)
mlp_tnr_url,mlp_recall_url,mlp_f1_url=pre_metri(model_mlp2_ze,pre_url_x_en,pre_url_y_en)
mlp_tnr_html,mlp_recall_html,mlp_f1_html=pre_metri(model_mlp_ze_html,pre_html_x_en,pre_html_y_en)

print('rf=======')
rf_tnr_com,rf_recall_com,rf_f1_com=pre_metri(model_rf1_ze,pre_x_en,pre_y_en)
rf_tnr_url,rf_recall_url,rf_f1_url=pre_metri(model_rf_ze_url,pre_url_x_en,pre_url_y_en)
rf_tnr_html,rf_recall_html,rf_f1_html=pre_metri(model_rf_ze_html,pre_html_x_en,pre_html_y_en)

print('lr=====')
lr_tnr_com,lr_recall_com,lr_f1_com=pre_metri(model_lr1_ze,pre_x_en,pre_y_en)
lr_tnr_url,lr_recall_url,lr_f1_url=pre_metri(model_lr_ze_url,pre_url_x_en,pre_url_y_en)
lr_tnr_html,lr_recall_html,lr_f1_html=pre_metri(model_lr_ze_html,pre_html_x_en,pre_html_y_en)

print('dt======')
dt_tnr_com,dt_recall_com,dt_f1_com=pre_metri(model_dt1_ze,pre_x_en,pre_y_en)
dt_tnr_url,dt_recall_url,dt_f1_url=pre_metri(model_dt_ze_url,pre_url_x_en,pre_url_y_en)
dt_tnr_html,dt_recall_html,dt_f1_html=pre_metri(model_dt_ze_html,pre_html_x_en,pre_html_y_en)

print('knn======')
knn_tnr_com,knn_recall_com,knn_f1_com=pre_metri(model_knn1_ze,pre_x_en,pre_y_en)
knn_tnr_url,knn_recall_url,knn_f1_url=pre_metri(model_knn2_ze,pre_url_x_en,pre_url_y_en)
knn_tnr_html,knn_recall_html,knn_f1_html=pre_metri(model_knn3_ze,pre_html_x_en,pre_html_y_en)

print('gb======')
gb_tnr_com,gb_recall_com,gb_f1_com=pre_metri(model_gb_ze,pre_x_en,pre_y_en)
gb_tnr_url,gb_recall_url,gb_f1_url=pre_metri(model_gb_ze_url,pre_url_x_en,pre_url_y_en)
gb_tnr_html,gb_recall_html,gb_f1_html=pre_metri(model_gb_ze_html,pre_html_x_en,pre_html_y_en)

print('ab======')
ada_tnr_com,ada_recall_com,ada_f1_com=pre_metri(model_ada_ze,pre_x_en,pre_y_en)
ada_tnr_url,ada_recall_url,ada_f1_url=pre_metri(model_ada_ze_url,pre_url_x_en,pre_url_y_en)
ada_tnr_html,ada_recall_html,ada_f1_html=pre_metri(model_ada_ze_html,pre_html_x_en,pre_html_y_en)

print('svm=======')
svm_tnr_com,svm_recall_com,svm_f1_com=pre_metri(model_svm_ze,pre_x_en,pre_y_en)
svm_tnr_url,svm_recall_url,svm_f1_url=pre_metri(model_svm_ze_url,pre_url_x_en,pre_url_y_en)
svm_tnr_html,svm_recall_html,svm_f1_html=pre_metri(model_ada_ze_html,pre_html_x_en,pre_html_y_en) 

print('cnn=====')
cnn_tnr_com,cnn_recall_com,cnn_f1_com=cnn_metri(model_cnn_combined,pre_x_re_en,pre_y_en)
cnn_tnr_url,cnn_recall_url,cnn_f1_url=cnn_metri(model_cnn_url,pre_url_x_re_en,pre_y_en)
cnn_tnr_html,cnn_recall_html,cnn_f1_html=cnn_metri(model_cnn_html,pre_html_x_re_en,pre_y_en)

    

mlp======
recall is 0.87, tnr is 0.99
recall is 0.88, tnr is 0.99
recall is 0.77, tnr is 0.89
rf=======
recall is 0.89, tnr is 1.00
recall is 0.89, tnr is 0.99
recall is 0.74, tnr is 0.98
lr=====
recall is 0.88, tnr is 1.00
recall is 0.88, tnr is 1.00
recall is 0.84, tnr is 0.83
knn======
recall is 0.76, tnr is 0.96
recall is 0.83, tnr is 0.97
recall is 0.73, tnr is 0.87
gb======
recall is 0.90, tnr is 0.99
recall is 0.89, tnr is 0.94
recall is 0.76, tnr is 0.94
svm=======
recall is 0.88, tnr is 1.00
recall is 0.88, tnr is 1.00
recall is 0.79, tnr is 0.88
cnn=====
recall is 0.89, tnr is 0.99
recall is 0.88, tnr is 0.99
recall is 0.69, tnr is 0.93
